In [47]:
import pandas as pd 
import matplotlib.pyplot as plt
import re 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [3]:
!pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 108.9 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 200.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     --------------------------- ---------- 41.0/57.6 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 606.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 660.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.5 MB 1.2 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 1.0 MB/s eta 0:00:02
   ------ -----


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
news_data = pd.read_csv("news.csv")

In [12]:
news_data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [13]:
news_data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [14]:
new_data = news_data.fillna('')

In [17]:
print(new_data.label.value_counts())

label
REAL    3171
FAKE    3164
Name: count, dtype: int64


In [18]:
news_data.replace({'label':{'REAL':0,'FAKE': 1}},inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11868\3158557054.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news_data.replace({'label':{'REAL':0,'FAKE': 1}},inplace=True)


In [19]:
news_data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [20]:
X= news_data.drop(['label'], axis=1)
Y= news_data['label']

In [21]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Lin

In [22]:
port_stem= PorterStemmer()

In [33]:
def stemming(title):
    stemmed_title = re.sub('[^a-zA-Z]',' ',title)
    stemmed_title = stemmed_title.lower()
    stemmed_title = stemmed_title.split()
    stemmed_title = [port_stem.stem(word) for word in stemmed_title if not word in stopwords.words('english')]
    stemmed_title = ' '.join(stemmed_title)
    return stemmed_title

In [34]:
news_data['title'] = news_data['title'].apply(stemming)

In [35]:
print(news_data['title'])

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: title, Length: 6335, dtype: object


In [37]:
#separating the data and label
X = news_data['title'].values
Y = news_data['label'].values

In [38]:
print(X)

['smell hillari fear'
 'watch exact moment paul ryan commit polit suicid trump ralli video'
 'kerri go pari gestur sympathi' ...
 'anti trump protest tool oligarchi inform'
 'ethiopia obama seek progress peac secur east africa'
 'jeb bush suddenli attack trump matter']


In [39]:
print(Y)

[1 1 0 ... 1 0 0]


In [40]:
Y.shape

(6335,)

In [41]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [42]:
X.shape

(6335, 6884)

In [44]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 45882 stored elements and shape (6335, 6884)>
  Coords	Values
  (0, 2230)	0.5606827254461214
  (0, 2817)	0.31703295849951846
  (0, 5631)	0.7649346276717717
  (1, 1212)	0.3538286538491304
  (1, 2101)	0.44595444447238664
  (1, 3939)	0.3279355637571237
  (1, 4462)	0.26757273402295384
  (1, 4630)	0.24403179029804972
  (1, 4905)	0.29382045650093463
  (1, 5263)	0.29163264316148546
  (1, 5933)	0.3501812302002013
  (1, 6315)	0.13561903129801023
  (1, 6571)	0.21736822482437437
  (1, 6671)	0.27855534550881683
  (2, 2536)	0.5345737509664292
  (2, 2576)	0.3357697230253524
  (2, 3335)	0.41740393299177314
  (2, 4428)	0.37615749278439464
  (2, 6015)	0.5345737509664292
  (3, 215)	0.41916879208304547
  (3, 561)	0.2969312816565521
  (3, 1764)	0.3507454498930073
  (3, 2048)	0.4448394256850793
  (3, 5956)	0.27803614912134916
  (3, 6288)	0.3148734995784654
  :	:
  (6330, 5809)	0.3024865273895357
  (6331, 4474)	0.5790025183139886
  (6331, 4493)	0

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [48]:
model = LogisticRegression()

In [49]:
model.fit(X_train, Y_train)

LogisticRegression()

In [50]:
# accuracy score on the training data
X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_train)

In [55]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9114048934490924


In [56]:
# accuracy score on the test data
X_test_predict = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)

In [59]:
print('Accuracy score of the test data : ', test_data_accuracy)


Accuracy score of the test data :  0.7947908445146015


In [64]:
X_new = X_test[100]

predict = model.predict(X_new)
print(predict)

if (predict[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [65]:
print(Y_test[3])

0
